## SAT-NAE

### Aim
Given a SAT problem, solve it such that for each clause, have at least one variable to be set to false.

### Example

We aim to solve a 3-SAT problem of the form

$(x_1 \lor x_2 \lor \neg x_3) \land (x_3 \lor \neg x_1 \lor \neg x_6) \land (\neg x_2 \lor x_4 \lor x_5) \land (\neg x_4 \lor \neg x_5 \lor x_6)$

such that each clause $C$ must satisfy that for the three variables in $C$, at least one is true, and at least one is false. By setting $w_i = (-1)^{x_i}$, we aim to solve:

$\displaystyle \max\limits_{w_i \in \{-1,1\}} \sum\limits_{c\in C}\frac{3}{4}-\frac{1}{4}(w_iw_j + w_iw_k + w_jw_k) \equiv \frac{3}{4}|C| - \frac{1}{4} \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k)$.

Thus, our problem becomes:

$\displaystyle \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k) + M\sum\limits_{i=1}^n (w_i w_{i+n})$

where the second sum accounts for a penalty where we want to ensure that $w_i$ and $\neg w_i$ indeed hold different values (and M is a sufficiently large positive number)

## Try for QAOA Form
$\sum\limits_{c \in C} (Z_iZ_j+Z_iZ_k+Z_jZ_k) + M\sum\limits_{i=1}^n(Z_iZ_{i+n})$

## Encode SAT Formula

- Number of variables = $n$
- Number of qubits that we will use: $2n$
- Encode the variables into their indices (i.e. $x_1 \implies 1$, $x_5 \implies 5$)
- Replace the negation of variable $i$ with ($i+n$)

### Example
- Input: (1 or 2 or not 3) and (not 1 or 2 or 3)
- Problem properties: $n = 3 \implies 2n = nqubits = 6$ 
- Output: $\left[[1,2,6], [4,2,3]\right]$

In [22]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import QAOA
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.providers.aer import AerSimulator
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import Session
import itertools

In [2]:
## Some constants
NEG_TEXT = "not "
AND_TEXT = " and "
OR_TEXT = " or "

## Parser Code

In [14]:
def parse_cnf():
    """Asks the user to input the number of variables used and the CNF itself, and parses the CNF

    Returns
    -------
    list(int)
        the parsed CNF according to the conventions we decide to use
    """
    num_variables = int(input("Enter the number of variables: "))
    cnf = input("Enter the CNF of the form above (ex: 0 1 2,0 n1 2): ")
    clauses = cnf.replace(",", " ")
    parsed_cnf = clauses.split(" ")
    for i in range(len(parsed_cnf)):
        if parsed_cnf[i].startswith("n"):
            parsed_cnf[i] = str(int(parsed_cnf[i][1:])+num_variables)
    parsed_cnf = list(map(lambda elem: int(elem), parsed_cnf))
    return parsed_cnf, num_variables

In [15]:
def parse_cnf_from_str(str_cnf, num_variables):
    clauses = str_cnf.replace(",", " ")
    parsed_cnf = clauses.split(" ")
    for i in range(len(parsed_cnf)):
        if parsed_cnf[i].startswith("n"):
            parsed_cnf[i] = str(int(parsed_cnf[i][1:])+num_variables)
    parsed_cnf = list(map(lambda elem: int(elem), parsed_cnf))
    return parsed_cnf, int(num_variables)

### Example

We aim to solve a 3-SAT problem of the form

$(x_1 \lor x_2 \lor \neg x_3) \land (x_3 \lor \neg x_1 \lor \neg x_6) \land (\neg x_2 \lor x_4 \lor x_5) \land (\neg x_4 \lor \neg x_5 \lor x_6)$

such that each clause $C$ must satisfy that for the three variables in $C$, at least one is true, and at least one is false. By setting $w_i = (-1)^{x_i}$, we aim to solve:

### General Case Formula

$\displaystyle \max\limits_{w_i \in \{-1,1\}} \sum\limits_{c\in C}\frac{3}{4}-\frac{1}{4}(w_iw_j + w_iw_k + w_jw_k) \equiv \frac{3}{4}|C| - \frac{1}{4} \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k)$.

Thus, our problem becomes:

$\displaystyle \min\limits_{w_i \in \{-1, 1\}} \sum\limits_{c\in C}(w_iw_j + w_iw_k + w_jw_k) + M\sum\limits_{i=1}^n (w_i w_{i+n})$

where the second sum accounts for a penalty where we want to ensure that $w_i$ and $\neg w_i$ indeed hold different values (and M is a sufficiently large positive number).

In this notebook, I chose `M = M_FACTOR*num_variables`

## Try for QAOA Form
$\sum\limits_{c \in C} (Z_iZ_j+Z_iZ_k+Z_jZ_k) + M\sum\limits_{i=1}^n(Z_iZ_{i+n})$

## Create the Problem Hamiltonian

In [16]:
from qiskit.opflow import I, Z

"""
Prepares a zero Hamiltonian of for num_variables qubits (i.e. a 2^n x 2^n zero matrix)
"""
def create_zero_hamiltonian(num_variables):
    zero_H = I-I # 2x2 0-matrix
    result = zero_H
    for i in range(2*num_variables-1):
        result = result^zero_H
    return result

"""
Creates the 3SAT-NAE problem Hamiltonian (to later be used by QAOA)
"""
def create_problem_hamiltonian(num_variables, problem_input, penalty_factor):
    h_p_p1 = create_zero_hamiltonian(num_variables)
    # Create first sum (triangle max-cut Hamiltonian per clause)
    for i in range(len(problem_input), 3):
        clause_H = create_zero_hamiltonian(num_variables)
        pairs = [
            (problem_input[i], problem_input[i+1]), 
            (problem_input[i], problem_input[i+2]), 
            (problem_input[i+1], problem_input[i+2])]
        for pair in pairs:
            pairH = 1
            for i in range(2*num_variables):
                if pair[0] == i or pair[1] == i:
                    pairH = pairH^Z
                else:
                    pairH = pairH^I
            clause_H += pairH
        h_p_p1 += clause_H

    # Create second sum (penalty term)
    h_p_p2 = create_zero_hamiltonian(num_variables)
    for i in range(num_variables):
        temp_h_p_p2 = 1
        for j in range(2*num_variables):
            if j == i or j == num_variables+i:
                temp_h_p_p2 = temp_h_p_p2^Z
            else:
                temp_h_p_p2 = temp_h_p_p2^I
        h_p_p2 += temp_h_p_p2
    h_p_p2 *= penalty_factor
    return h_p_p1 + h_p_p2

## Verify the consistency and problem constraint satisfactions

In [5]:
"""
Verifies if the variable assignments are consistent (i.e. make sure that x and not(x) evaluate to different values)
Returns a boolean (True if the assignments are consistent, False otherwise)
"""
def verify_consistency(assignments, num_variables, show_info=True):
    for i in range(num_variables):
        # Check if the assignment to x and indeed not x are the same
        if assignments[i] == assignments[i+num_variables]:
            return False
    return True

In [6]:
"""
Verifies if the formula is satsified or not (with additional checks for the NAE constraint).
Returns a boolean (True if both constraints are satisfied, False otherwise)
"""
def verify(result, sat_nae, num_variables):
    clause_values = []
    nae_holds = True
    for clause_index in range(len(sat_nae)):
        clause_value = False
        clause_and = True # do an AND inside each clause to check for NAE
        for i in range(3): # Because 3 variables per clause
            if sat_nae[clause_index][i] < num_variables:
                result[sat_nae[clause_index][i]]
                clause_value = clause_value or bool(result[sat_nae[clause_index][i]])
                clause_and = clause_and and bool(result[sat_nae[clause_index][i]])
            else:
                clause_value = clause_value or not bool(result[sat_nae[clause_index][i]-num_variables])
                clause_and = clause_and and bool(result[sat_nae[clause_index][i]])
        clause_values.append(clause_value)
        if clause_and:
            nae_holds = False
    final_value = True
    nae_value = True
    for i in range(len(clause_values)):
        final_value = final_value and clause_values[i]
    if final_value and nae_holds:
        return True
    else:
        return False

In [17]:
# Cell copied from https://qiskit.org/documentation/tutorials/algorithms/05_qaoa.html
from collections import OrderedDict
from qiskit.opflow import StateFn
import numpy as np


def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    elif isinstance(state_vector, StateFn):
        binary_string = list(state_vector.sample().keys())[0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x
    else:
        n = int(np.log2(state_vector.shape[0]))
        k = np.argmax(np.abs(state_vector))
        x = np.zeros(n)
        for i in range(n):
            x[i] = k % 2
            k >>= 1
        return x

## Input CNF
A 3SAT CNF is of the form $(x_1 \lor x_2 \lor x_3) \land (\neg x_2 \lor x_3 \lor x_4)$ for example

In this notebook, we will assume that the cnf variables ($x_1, x_2, ...$) are given as integers (starting from 0) instead of strings, i.e.
$x_1 \mapsto 0$, $x_2 \mapsto 1$, etc... This is not very intuitive but it makes parsing easier. We might change it later.

- $\neg$ operator: The negation is denoted by an `n` in front of a particular variable $\implies$ Ex: $\neg x_2 \mapsto$ `not 2`
- $\lor$ operator: The $\lor$ operator is denoted by the ` or ` keyword. $\implies$ Ex: $x_1 \lor \neg x_2 \lor x_3 \mapsto$ `(0 or not 1 or 2)` 
- $\land$ operator: The $\land$ operator is denoted by the ` and ` keyword between clauses $\implies$ Ex: $(x_1 \lor \neg x_2 \lor x_3) \land (\neg x_1 \lor \neg x_2 \lor \neg x_3) \mapsto$ `(0 or not 1 or 2) and (not 0 or not 1 or not 2)`

Therefore, for example, given the following CNF:

$(x_1 \lor x_2 \lor \neg x_3) \land (x_3 \lor \neg x_1 \lor \neg x_6) \land (\neg x_2 \lor x_4 \lor x_5) \land (\neg x_4 \lor \neg x_5 \lor x_6)$

the input CNF to this program will be formulated as

`(0 or 1 or not 2) and (2 or not 0 or not 5) and (not 1 or 3 or 4) and (not 3 or not 4 or 5)` (as a string)

In [10]:
num_variables = int(input("Enter the number of variables in the formula: "))
formula = input("Enter a CNF (example: (0 or 1 or not 2) and (not 0 or 3 or 4): ")
# example: (0 or 1 or not 2) and (2 or not 0 or not 5) and (not 1 or 3 or 4) and (not 3 or not 4 or 5)
# example: (0 or 1 or 2) and (not 0 or 1 or 2) and (0 or not 1 or 2) and (0 or 1 or not 2)

sat_nae = parse_formula(formula, num_variables)
print(sat_nae)
M = 30 # penalty factor

[[0, 1, 8], [2, 6, 11], [7, 3, 4], [9, 10, 5]]


In [12]:
from qiskit.algorithms import NumPyMinimumEigensolver
h_p = create_problem_hamiltonian(num_variables, sat_nae, M)
classical_eigensolver = NumPyMinimumEigensolver()
print(h_p)

0.0 * IIIIIIIIIIII
+ 0.0 * IIIIIIIIIIII
+ 1.0 * ZZIIIIIIIIII
+ 1.0 * ZIIIIIIIZIII
+ 1.0 * IZIIIIIIZIII
+ 0.0 * IIIIIIIIIIII
+ 1.0 * IIZIIIZIIIII
+ 1.0 * IIZIIIIIIIIZ
+ 1.0 * IIIIIIZIIIIZ
+ 0.0 * IIIIIIIIIIII
+ 1.0 * IIIZIIIZIIII
+ 1.0 * IIIIZIIZIIII
+ 1.0 * IIIZZIIIIIII
+ 0.0 * IIIIIIIIIIII
+ 1.0 * IIIIIIIIIZZI
+ 1.0 * IIIIIZIIIZII
+ 1.0 * IIIIIZIIIIZI
+ 0.0 * IIIIIIIIIIII
+ 30.0 * ZIIIIIZIIIII
+ 30.0 * IZIIIIIZIIII
+ 30.0 * IIZIIIIIZIII
+ 30.0 * IIIZIIIIIZII
+ 30.0 * IIIIZIIIIIZI
+ 30.0 * IIIIIZIIIIIZ


In [18]:
# Get the theoretical Max-Cut value according to the previous formula
def get_theoretical_maxcut_value(G, parsed_cnf, num_variables, M):
    """Compute the theoretical max-cut value
    
    Parameters
    ----------
    G : nx.Graph
        the graph whose max-cut to compute
    parsed_cnf : list(int)
        the parsed cnf containing values in [0, 2N-1]
    num_variables: int
        the number of variables used in the CNF (= N)
    M : int
        the penalty factor for consistency

    Returns
    -------
    float
        the theoretical max-cut value
    """
    # cnf_contribution is the second part of the MaxCutValue_theory above
    cut_value = 0
    for u,v in G.edges:
        cut_value += G[u][v]["weight"]/2
        
    for i in range(num_variables):
        if i in parsed_cnf and i + num_variables in parsed_cnf:
            cut_value += M/2
    cut_value += len(parsed_cnf)/6
    return cut_value


## Run QAOA

In [ ]:
service = QiskitRuntimeService()
method = "statevector"
device = "CPU"
backend = AerSimulator(method=method, device=device)
session = Session(backend=backend)
service = QiskitRuntimeService()

# For testing
num_successes = 0
num_runs = 20

# Running using Qiskit Runtime
qaoa = QAOA(optimizer=COBYLA(maxiter=1000), quantum_instance=backend, reps=3)
output = None

# Compute theoretical GSE
num_negated_variables = 0
flattened_cnf = list(itertools.chain.from_iterable(sat_nae))
for i in range(num_variables):
    if i in flattened_cnf and i+num_variables in flattened_cnf:
        num_negated_variables += 1
ground_state_value = -len(sat_nae) -M*num_negated_variables

print(f"Theoretical ground state energy: {ground_state_value}")

for i in range(num_runs):
    with Session(service=service,backend=backend) as session:
        output = qaoa.compute_minimum_eigenvalue(h_p)
    eigenstate = list(map(lambda x: int(x), list(sample_most_likely(output.eigenstate))[::-1]))
    eigenvalue = output.eigenvalue.real
    if verify_consistency(eigenstate, num_variables) and verify(eigenstate, sat_nae, num_variables):
        print(f"Run {i+1}: Success (eigval = {eigenvalue}, eigvec = {eigenstate})")
        num_successes += 1
    else:
        print(f"Run {i+1}: Failure (eigval = {eigenvalue}, eigvec = {eigenstate})")

print(f"Success Rate: {100*num_successes/num_runs}%")

In [19]:
def calculate_error(parsed_cnf, num_variables, result):
    """Calculate the NAE-3SAT error

    Parameters
    ----------
    parsed_cnf: list(int)
        the initial parsed CNF
    num_variables: int
        the number of variables used (without distinguishing x and not x)
    result: list(int)
        the resulting variable assignments

    Returns
    int, int
        the NAE-SAT error and the consistency error, respectively
    """
    
    nae_sat_error = 0
    consistency_error = 0
    # Iterate over the clauses of the CNF
    for i in range(0, len(parsed_cnf), 3):
        v1 = result[parsed_cnf[i]]
        v2 = result[parsed_cnf[i+1]]
        v3 = result[parsed_cnf[i+2]]
        # if clause is satisfied but not SAT-NAE, or if the clause is not satisfied, we add 1 to nae_sat_error
        if v1 + v2 + v3 == 3 or v1 + v2 + v3 == 0:
            nae_sat_error += 1
    
    for i in range(num_variables):
        # if a variable is the same as its negation, we add 1 to consistency_error
        if  i + num_variables < len(result) and result[i] == result[i+num_variables]:
            consistency_error += 1
    return nae_sat_error, consistency_error

In [25]:
def compute_min_M_value(parsed_cnf):
    """Computes the minimal required M value for a CNF

    Parameters
    ----------
    parsed_cnf: list(int)
        the input CNF
    
    Returns
    -------
    int
        the minimal penalty factor M
    """

    pair_appearances = {}
    for i in range(0, len(parsed_cnf), 3):
        v_1 = parsed_cnf[i]
        v_2 = parsed_cnf[i+1]
        v_3 = parsed_cnf[i+2]
    pair_appearances[(v_1,v_2)] = 1 if (v_1, v_2) not in pair_appearances else pair_appearances[(v_1, v_2)] + 1
    pair_appearances[(v_1,v_3)] = 1 if (v_1, v_3) not in pair_appearances else pair_appearances[(v_1, v_3)] + 1
    pair_appearances[(v_2,v_3)] = 1 if (v_2, v_3) not in pair_appearances else pair_appearances[(v_2, v_3)] + 1
    max_appearance = 0
    for key in pair_appearances:
        if pair_appearances[key] > max_appearance:
            max_appearance = pair_appearances[key]
    return max_appearance + 1

In [43]:
def predict_result_for_nae3sat_cnfs(nae_filename, runs_per_cnf):
    """Reads and solves NAE-3SAT CNFs from a file, outputs the success rate per CNF

    Parameters
    ----------
    nae_filename: str
        the name of the file containing NAE-3SAT CNFs
    runs_per_cnf: int
        the number of QAOA runs to do per CNF
    """

    service = QiskitRuntimeService()
    
    nae_file = open(nae_filename, "r")
    nae_cnfs = list(map(lambda elem: elem.replace("\n", ""), nae_file.readlines()))
    nae_file.close()
    num_nae_cnfs = len(nae_cnfs)
    method = "statevector"
    device = "CPU"
    backend = AerSimulator(method=method, device=device)
    qaoa = QAOA(optimizer=COBYLA(maxiter=1000), quantum_instance=backend, reps=1)

    for cnf_index in range(len(nae_cnfs)):
        [num_vars_str, cnf] = nae_cnfs[cnf_index].split(" -- ")
        num_vars = int(num_vars_str)
        parsed_cnf, _ = parse_cnf_from_str(cnf, num_vars)
        num_clauses = int(len(parsed_cnf)/3)
        M = compute_min_M_value(parsed_cnf)
        hamiltonian = create_problem_hamiltonian(num_vars, parsed_cnf, M)     
        
        individual_success_rate = 0
        for i in range(runs_per_cnf):
            output = None
            with Session(service=service,backend=backend):
                output = qaoa.compute_minimum_eigenvalue(hamiltonian)
            eigenstate = list(map(lambda x: int(x), list(sample_most_likely(output.eigenstate))))[::-1] # not reversing here
            nae_sat_error, consistency_error = calculate_error(parsed_cnf, num_vars, eigenstate)
            if nae_sat_error + consistency_error == 0:
                individual_success_rate += 1
        print(f"Success Rate For ({num_vars}, {num_clauses}): {100*individual_success_rate/runs_per_cnf}")
        print(f"Progress: {cnf_index+1}/{num_nae_cnfs}")

In [ ]:
predict_result_for_nae3sat_cnfs("nae3sat_cnfs.txt", 10)